In [2]:
import cv2
from PIL import Image
import numpy as np
from scipy import ndimage
from skimage.feature import peak_local_max
from skimage.morphology import watershed
import matplotlib.pyplot as plt
import sys
import math
from libsvm.svmutil import *

In [3]:
def checkCoords(rows, cols, fIx, fIy, sIx, sIy):
    return fIx >= 0 and fIy >= 0 and fIx <= (rows - 1) and fIy <= (cols - 1) and sIx >= 0 and sIy >= 0 and sIx <= (rows - 1) and sIy <= (cols - 1)

In [4]:
np.set_printoptions(threshold=sys.maxsize)

In [5]:
def calculateCooccurrenceMatrix(grayImg, binNumber, di, dj):
    rows, cols = grayImg.shape
    bins = {}
    for r in range(rows):
        for c in range(cols):
            fIx = r + di
            fIy = c + dj
            sIx = r + 1 + di
            sIy = c + 1 + dj
            if not checkCoords(rows, cols, fIx, fIy, sIx, sIy):
                continue
            fpx = grayImg[fIx][fIy]
            spx = grayImg[sIx][sIy]
            if (fpx, spx) in bins:
                bins[(fpx, spx)] += 1
            else:
                bins[(fpx, spx)] = 1
            
    
    arr2Img = np.zeros(shape = (256,256))
    for key, val in bins.items():
        arr2Img[key[0]][key[1]] = val
    
    im = Image.fromarray(np.uint8(arr2Img))
    #plt.imshow(im)
    #plt.show()
    return arr2Img
def calculateAccumulatedCooccurrenceMatrix(grayImg, binNumber, d):
    dists = [(d, 0), (d, d), (0 , d), (-d, d), (-d, 0), (-d, -d), (0, -d), (d, -d)]
    arr = np.zeros(shape=(256,256))
    for d in dists:
        arr += calculateCooccurrenceMatrix(grayImg, binNumber, d[0], d[1])
    im = Image.fromarray(np.uint8(arr))
    return arr
def calculateCooccurrenceFeatures(accM):
    accMNormalized = normalizeAccm(accM)
    asm = angularSecondMoment(accMNormalized)
    entr = entropy(accMNormalized)
    cont = contrast(accMNormalized)
    invDiff = inverseDiffMoment(accMNormalized)
    maxProb = maxProbability(accMNormalized)
    corr = correlation(accMNormalized)
    return asm, entr, cont, invDiff, maxProb, corr
def normalizeAccm(accM):
    rows, cols = accM.shape
    puv = 0
    for r in range(rows):
        for c in range(cols):
            puv += accM[r][c]
    
    if puv == 0:
        puv = 1
    for r in range(rows):
        for c in range(cols):
            accM[r][c] /= puv
            #if accM[r][c] > 0:
                #print(str(accM[r][c]))
    im = Image.fromarray(np.uint8(accM))
    #plt.imshow(im)
    #plt.show()
    return accM
def angularSecondMoment(accMNorm):
    rows, cols = accMNorm.shape
    asm = 0
    for r in range(rows):
        for c in range(cols):
            asm += math.pow(accMNorm[r][c], 2)
    return asm
def entropy(accMNorm):
    rows, cols = accMNorm.shape
    entr = 0
    for r in range(rows):
        for c in range(cols):
            if accMNorm[r][c] == 0:
                continue
            entr += accMNorm[r][c] * math.log(accMNorm[r][c])
    return -entr
def contrast(accMNorm):
    rows, cols = accMNorm.shape
    cont = 0
    for r in range(rows):
        for c in range(cols):
            cont += accMNorm[r][c] * (r-c)*(r-c)
    return cont
def inverseDiffMoment(accMNorm):
    rows, cols = accMNorm.shape
    idm = 0
    for r in range(rows):
        for c in range(cols):
            idm += accMNorm[r][c] / (1 + math.pow((r - c), 2))
    return idm
def maxProbability(accMNorm):
    rows, cols = accMNorm.shape
    maxProb = 0
    for r in range(rows):
        for c in range(cols):
            if accMNorm[r][c] > maxProb:
                maxProb = accMNorm[r][c]
    return maxProb
def correlation(accMNorm):
    rows, cols = accMNorm.shape
    Ux = []
    Uy = []
    for r in range(rows):
        ux = 0
        for c in range(cols):
            ux += accMNorm[r][c]
        Ux.append(ux)
    for c in range(cols):
        uy = 0
        for r in range(rows):
            uy += accMNorm[r][c]
        Uy.append(uy)
    
    sigX = []
    sigY = []
    for r in range(rows):
        sigx = 0
        for c in range(cols):
            sigx += math.pow((r - Ux[r]), 2)*accMNorm[r][c]
        sigX.append(math.sqrt(sigx))
    for c in range(cols):
        sigy = 0
        for r in range(rows):
            sigy += math.pow((r - Uy[c]), 2)*accMNorm[r][c]
        sigY.append(math.sqrt(sigy))
    
    
    corr = 0
    for r in range(rows):
        for c in range(cols):
            if sigX[r] == 0 or sigY[c] == 0:
                continue
            corr += ( (r-Ux[r]) / sigX[r] ) * ( (c-Uy[c]) / sigY[c] ) * accMNorm[r][c]
    return corr


In [6]:
import glob
imgs_train = []
imgs_test = []
labels_train = []
labels_test = []
for img in glob.glob("C:/Users/etuna/Desktop/HW2/dataset/dataset/training/*"):
    n= cv2.imread(img)
    imgs_train.append(n)
    
for img in glob.glob("C:/Users/etuna/Desktop/HW2/dataset/dataset/test/*"):
    n= cv2.imread(img)
    imgs_test.append(n)
    
with open('C:/Users/etuna/Desktop/HW2/dataset/dataset/training_labels.txt', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_train = list(map(int,labelStr))
    
with open('C:/Users/etuna/Desktop/HW2/dataset/dataset/test_labels.txt', 'r') as file:
    labelStr = file.read().replace('\n', '')
    labels_test = list(map(int,labelStr))
    
print("Train set size:",len(imgs_train))
print("Test set size:",len(imgs_test))
print("Train Labels size:", len(labels_train))
print("Train Labels size:", len(labels_test))


Train set size: 186
Test set size: 144
Train Labels size: 186
Train Labels size: 144


In [7]:
def prepareData(N,data):
    feat_data = []
    for i,img in enumerate(data):
        print("Img :",i)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        accM = calculateAccumulatedCooccurrenceMatrix(gray ,8, 10)
        subimgs = getCroppedImages(N, accM)
        A=0
        B=0
        C=0
        D=0
        E=0
        F=0
        for subim in subimgs:
            a,b,c,d,e,f = calculateCooccurrenceFeatures(subim)
            A += a
            B += b
            C += c
            D += d
            E += e
            F += f
        lenn = len(subimgs)
        feat_data.append([A/lenn,B/lenn,C/lenn,D/lenn,E/lenn,F/lenn])
    return feat_data


In [8]:
def getCroppedImages(N,img):
    imgs = []
    rows, cols = img.shape
    rPointer = 0
    cPointer = 0
    while not cPointer >= cols:
        while not rPointer  >= rows:
            im = img[rPointer:rPointer+4, cPointer:cPointer+4]
            imgs.append(im)
            rPointer += N
        cPointer += N
    return imgs
    

In [10]:
feat_train = prepareData(64,imgs_train)
feat_test = prepareData(64,imgs_test)
print("Length of feat_train:", str(len(feat_train)))
print("Length of feat_test:", str(len(feat_test)))

Img : 0
Img : 1
Img : 2
Img : 3
Img : 4
Img : 5
Img : 6
Img : 7
Img : 8
Img : 9
Img : 10
Img : 11
Img : 12
Img : 13
Img : 14
Img : 15
Img : 16
Img : 17
Img : 18
Img : 19
Img : 20
Img : 21
Img : 22
Img : 23
Img : 24
Img : 25
Img : 26
Img : 27
Img : 28
Img : 29
Img : 30
Img : 31
Img : 32
Img : 33
Img : 34
Img : 35
Img : 36
Img : 37
Img : 38
Img : 39
Img : 40
Img : 41
Img : 42
Img : 43
Img : 44
Img : 45
Img : 46
Img : 47
Img : 48
Img : 49
Img : 50
Img : 51
Img : 52
Img : 53
Img : 54
Img : 55
Img : 56
Img : 57
Img : 58
Img : 59
Img : 60
Img : 61
Img : 62
Img : 63
Img : 64
Img : 65
Img : 66
Img : 67
Img : 68
Img : 69
Img : 70
Img : 71
Img : 72
Img : 73
Img : 74
Img : 75
Img : 76
Img : 77
Img : 78
Img : 79
Img : 80
Img : 81
Img : 82
Img : 83
Img : 84
Img : 85
Img : 86
Img : 87
Img : 88
Img : 89
Img : 90
Img : 91
Img : 92
Img : 93
Img : 94
Img : 95
Img : 96
Img : 97
Img : 98
Img : 99
Img : 100
Img : 101
Img : 102
Img : 103
Img : 104
Img : 105
Img : 106
Img : 107
Img : 108
Img : 109
Img : 110


In [12]:
feat_train_temp = feat_train
feat_test_temp = feat_test


[[nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan

In [13]:
def balanceTrainingSets(data, labels):
    mdata = normalizeFeatures(data)
    label_dict = {}
    for l in labels:
        if l in label_dict:
            label_dict[l] += 1
        else:
            label_dict[l] = 1
    minEl = 9999
    minLabel = -1
    for k, v in label_dict.items():
        if v < minEl:
            minEl = v
            minLabel=k
            
    for k, v in label_dict.items():
        if k == minLabel:
            label_dict[k] = minEl
        else:
            label_dict[k] = minEl + (label_dict[k]-minEl)/2

    newData = []
    newLabels = []
    
    for i, d in enumerate(mdata):
        label= labels[i]
        if label_dict[label] > 0:
            newData.append(d)
            newLabels.append(label)
            label_dict[label] -= 1
    return newData, newLabels
            
    

In [14]:
def normalizeFeatures(features):
    #print(features)
    mins = np.min(features, axis = 0)
    maxs = np.max(features, axis = 0)
    newFeatures = []
    for i, f in enumerate(features):
        mf = (f - mins)/(maxs-mins)
        #print(f)
        #print(mins)
        #print(maxs)
        #print(mf)
        #print("\n\n\n")
        newFeatures.append(mf)
    #print("newfeats:")
    #print(newFeatures)
    return features
        

In [15]:
def getClassAccuricies(result, actual):
    correctDict = {}
    for i, r in enumerate(result):
        actVal = actual[i]
        if actVal == r:
            if actVal in correctDict:
                correctDict[actVal] += 1
            else:
                correctDict[actVal] = 1

    actualDict = {}
    for a in actual:
        if a in actualDict:
            actualDict[a] += 1
        else:
            actualDict[a] = 1
    mres = {}
    for r in correctDict:
        corr = correctDict[r]
        act = actualDict[r]
        mres[r] = (corr/act)
        
    return mres
        

In [42]:
feat_train

[[nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan, nan, nan, 0.0, nan],
 [nan, nan

In [41]:
feat_train_balanced, labels_training_balanced = balanceTrainingSets(feat_train, labels_train)
prob = svm_problem(labels_training_balanced,feat_train_balanced)
param = svm_parameter()
param.C=1000
param.kernel_type=LINEAR
model = svm_train(prob, param)
svm_save_model("LINEAR_C1_mdl",model)
result = svm_predict(labels_train, feat_train, model)
result = svm_predict(labels_test, feat_test, model)
mres=getClassAccuricies(result[0], labels_test)
print("Results :", mres)

Accuracy = 20.4301% (38/186) (classification)
Accuracy = 27.0833% (39/144) (classification)
Results : {3: 1.0}


<ipython-input-14-9ab7ec2dc06e>:7: RuntimeWarning: invalid value encountered in true_divide
  mf = (f - mins)/(maxs-mins)
